# Cubeviz Demonstration Notebook

This notebook demonstrates the Cubeviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Cubeviz, can be found here: https://jdaviz.readthedocs.io/en/latest/cubeviz/

Import modules needed for this notebook.

In [ ]:
import warnings
import tempfile

from astroquery.mast import Observations
from photutils.aperture import CircularAperture
from regions import PixCoord, CirclePixelRegion

from jdaviz import Cubeviz

We create a Cubeviz instance and show it.

In [ ]:
cubeviz = Cubeviz()
cubeviz.show()

Here we download a JWST MIRI cube from [MAST](https://masttest.stsci.edu/) using `astroquery`, 
to use as our example data. By default the downloaded file goes to your
temp directory, and thus may eventually be garbage collected (deleted) by your system.
If you would like to have the file permanently, simply uncomment the second line below
and provide the directory path on your system where you would like the file stored.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. You can turn off caching by setting `cache=False` in the `download_file` call to
force it to re-download the file if desired.

In [ ]:
data_dir = tempfile.gettempdir()
#data_dir = "/User/username/Data/"

fn = "jw02732-o004_t004_miri_ch1-shortmediumlong_s3d.fits"
result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(f'{data_dir}/{fn}')

Alternatively, the data and the configuration can be autodetected and loaded simultaneously by calling `jdaviz.open(f'{data_dir}/{fn}')`

The upper left viewer containing the FLUX cube is the default viewer that is accessible as follows.

In [ ]:
viewer = cubeviz.default_viewer

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on an image viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = cubeviz.get_interactive_regions()

In [ ]:
regions

It is also possible to programmatically pass a `regions` shape or a `photutils` aperture shape.

*NOTE: Sky regions are not yet supported for data cubes.*

In [ ]:
data = cubeviz.get_data('jw02732-o004_t004_miri_ch1-shortmediumlong_s3d.fits[SCI]')

In [ ]:
# photutils aperture
my_aper = CircularAperture((15, 10), r=5)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=35, y=35), radius=10)

my_regions = [my_aper, my_reg]
cubeviz.load_regions(my_regions)

To get the spectra:

In [ ]:
spectra_dict = cubeviz.specviz.get_spectra(apply_slider_redshift=True)
spectra_dict

This contains the masked spectrum extracted from "Subset 1" region.

In [ ]:
subset1_spec1d = spectra_dict['jw02732-o004_t004_miri_ch1-shortmediumlong_s3d.fits[SCI] (Subset 1)']
subset1_spec1d